# 実行環境の作成

## Google Cloud Storage

下記コードでGCPに接続

In [9]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [11]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/09/14 00:14:37.895541 Start gcsfuse/0.41.6 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/09/14 00:14:37.913177 Opening GCS connection...
2022/09/14 00:14:38.270803 Mounting file system "statistics-hyogo"...
2022/09/14 00:14:38.271397 File system has been successfully mounted.


## Google Secret Maneger

ライブラリをインストール

In [ ]:
pip install google-cloud-secret-manager

シークレットを呼び出す関数

In [2]:
from google.cloud import secretmanager


def access_secret(project_id, secret_name, version='latest'):
    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path(project_id, secret_name, version)
    response = client.access_secret_version(request={"name":name})
    payload = response.payload.data.decode("UTF-8")
    return payload

ESTAT_APPIDを取得する

In [ ]:
PROJECT_ID = 'primal-buttress-342908'

ESTAT_APPID = access_secret(PROJECT_ID,'ESTAT_APPID' )

print(ESTAT_APPID)

## estat-APIのデータを取得

都道府県リストを定義

都道府県コードのリストを返却する関数　['01000','02000',,,,,]

In [ ]:
def prefCodes():
  prefCodes = [d.get('prefCode') for d in prefList['result']]
  return [f'{n:02}'+'000' for n in prefCodes]


### estat-APIからデータ取得

ESTAT-APPIDは環境変数で管理する

In [ ]:
# APP_ID
ESTAT_APPID = '4425fca933c4a0aea169978c4c07cc35ecbbef62'

パラメータの設定

とりあえず総人口とする

In [ ]:
params = {
    'statsDataId':'0000010101',
    'cdCat01':'A1101',
}

estat-APIのデータを取得する

In [ ]:
import urllib.parse
import urllib.request
import json

# paramsにappIdを追加
params['appId']=ESTAT_APPID

# paramsにcdArea（全都道府県）を追加
params['cdArea']=",".join(prefCodes())

# リクエストURLの生成
url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
url += urllib.parse.urlencode(params)
print(url)

# レスポンスを取得
with urllib.request.urlopen(url) as response:
    res = json.loads(response.read().decode('utf-8'))
    print(res)

NameError: ignored

関数にしてみる

In [ ]:
def get_estat_response_prefecture(params):
  import urllib.parse
  import urllib.request
  import json

  # appId
  params['appId']=ESTAT_APPID
  
  # 都道府県コードを設定
  params['cdArea']=",".join(prefCodes())

  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(params)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

response = get_estat_response_prefecture(params)
print(response)

NameError: ignored

### pandas DataFrameに変換

VALUEをDataFrameに変換

In [ ]:
import pandas as pd

res = get_estat_response_prefecture(params)

# DATA_INF
data = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

df = pd.json_normalize(data, sep='_')
print(df)

CLASS_OBJをDataFrameに変換

例えば'@id'=='cat01'の場合

In [ ]:
# CLASS_INF
CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
cat01 = [d.get('CLASS') for d in filter(lambda x: x['@id'] == 'cat01', CLASS_OBJ )]

df_cat01 = pd.json_normalize(cat01)
print(df_cat01)

   @code      @name @level @unit
0  A1101  A1101_総人口      1     人


'@level'と'@unit'は不要なので削除する。

これで残るのは'@code'と'@name'

In [ ]:
# 不要な列を削除
df_1=df_cat01.copy().drop(columns=['@level', '@unit'])

print(df_1)

   @code      @name
0  A1101  A1101_総人口


逆に、'@code'と'@name'だけ抽出すると書いた方が明示的でわかりやすい

In [ ]:
df_2 = df_cat01.copy()[['@code','@name']]
print(df_2)

   @code      @name
0  A1101  A1101_総人口


CLASS_OBJの'@id'には'cat01'や'time'があるので、VALUEのDataFrameと一気に外部結合してしまう。

In [ ]:
# CLASS_INF
CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

# VALUE
VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

df_value = pd.json_normalize(VALUE)

for d in CLASS_OBJ :
  # キー名を取得 @time,@area,,,
  key_name = '@{}'.format(d['@id'])

  # DataFrameに変換
  df_class = pd.json_normalize(d['CLASS']) 

  # 不要な列を削除
  if '@level' in df_class.columns :
    df_class = df_class.drop(columns='@level')
  if '@unit' in df_class.columns :
    df_class = df_class.drop(columns='@unit')

  # @nameの不要な情報（A1101等）を削除
  # df_class['@name'] = df_class.apply(lambda x: x['@name'].replace(x['@code']+'_', ''), 1)

  # @codeをキー名に変更
  df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

  # df_valueとdf_classを結合
  df_value = pd.merge(df_value, df_class, on=key_name, how='outer')

print(df_value)

ここまでを関数化するとこんな感じ

In [ ]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # VALUE
  VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  df_value = pd.json_normalize(VALUE)

  for d in CLASS_OBJ :
    # キー名を取得 @time,@area,,,
    key_name = '@{}'.format(d['@id'])

    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_value = pd.merge(df_value, df_class, on=key_name, how='outer')
  
  return df_value


df_org = conv_estat_response_to_dataframe(response)
print(df_org)


### DataFrameの整形

使い勝手が良くなるように、DataFrameを整形していく

必要な列だけ抽出する

In [ ]:
# 特定の列だけ抽出
df_3 = df_org[['@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

print(df_3)

列名を変更する

In [ ]:
# 列名を変更
columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
df_3 = df_3.rename(columns=columns)

print(df_3)

欠損値が含まれる行を削除する

In [ ]:
df_3 = df_3.dropna()

print(df_3)

categoryNameから不要な情報（categoryCode）を削除する

In [ ]:
df_3['categoryName'] = df_3.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

print(df_3)

timeCodeを4桁文字列に置換する

In [ ]:
df_3['timeCode'] = df_3.apply(lambda x: x['timeCode'][:4], 1)

print(df_3)

In [ ]:


# 列名から@を削除
df_value = df_value.rename(columns=lambda x: x.replace('@', ''))
df_value = df_value.rename(columns={'$': 'value'})
# 欠損値を削除
df_value = df_value.dropna()
df_value['value'] = df_value['value'].astype(int)

# 特定の列だけ抽出
df_value = df_value[['cat01','cat01_name','time','time_name','area','area_name','value','unit']]

# 列名を変更
columns = {'cat01':'categoryCode','cat01_name':'categoryName','time':'timeCode','time_name':'timeName','area':'areaCode','area_name':'areaName'}
df_value = df_value.rename(columns=columns)

print(df_value)

DataFrameから'timeCode'の列を抽出して、重複を除いたリストを作成

In [ ]:

# 年次一覧リストを作成
times = df_value['timeCode'].tolist()
times = list(set(times))
print(len(times))

# 最終DataFrame
cols = ['categoryCode', 'categoryName','timeCode','timeName','areaCode','areaName','value','unit','rank']
df_last = pd.DataFrame(index=[], columns=cols)
print(df_last)

# 順位を付与して、最終DataFrameに整形する
for time in times:
  df = df_value.copy()[df_value['timeCode'] == time]
  df['rank'] = df.rank(ascending=False)['value'].astype(int).astype(str)
  df['value'] = df['value'].astype(str)
  df_last = pd.concat([df_last, df])

print(df_last)

## 統計データの処理

In [ ]:
params = {
    'statsDataId':'0000010101',
}

In [ ]:
def get_estat():
  import urllib.parse
  import urllib.request
  import json


  params = {
    'appId':ESTAT_APPID,
  }

  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsList?'
  url += urllib.parse.urlencode(params)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

response = get_estat_response_prefecture(params)
print(response)

{'GET_STATS_DATA': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-09-13T21:03:34.326+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0000010101', 'NARROWING_COND': {'CODE_CAT01_SELECT': 'A1101', 'CODE_AREA_SELECT': '01000,02000,03000,04000,05000,06000,07000,08000,09000,10000,11000,12000,13000,14000,15000,16000,17000,18000,19000,20000,21000,22000,23000,24000,25000,26000,27000,28000,29000,30000,31000,32000,33000,34000,35000,36000,37000,38000,39000,40000,41000,42000,43000,44000,45000,46000,47000'}, 'DATA_FORMAT': 'J', 'START_POSITION': 1, 'METAGET_FLG': 'Y'}, 'STATISTICAL_DATA': {'RESULT_INF': {'TOTAL_NUMBER': 2115, 'FROM_NUMBER': 1, 'TO_NUMBER': 2115}, 'TABLE_INF': {'@id': '0000010101', 'STAT_NAME': {'@code': '00200502', '$': '社会・人口統計体系'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '都道府県データ 基礎データ', 'TITLE': {'@no': '0000010101', '$': 'Ａ\u3000人口・世帯'}, 'CYCLE': '年度次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2022-03-04', 'SMALL_AREA': 0, 'MAIN_CATEGORY': 

Colab ノートブックには、Google ドライブ アカウント（スプレッドシートを含む）からご自分のデータをインポートできます。また、GitHub やその他多くのソースからのインポートも可能です。データのインポートについて、またデータ サイエンスで Colab を使用する方法の詳細については、<a href="#working-with-data">データの操作</a>の下にあるリンクをクリックしてください。